### The Vanishing Gradient Problem

Even though RNNs cover the memory problem of simple neural networks, in practice we still face issues. The introduced Simple RNNs do have problems with long-term dependencies which they seem to forget.

Let us have a look at the following sentence as a sequence.

##### “In France, I had a great time and I learnt some of the ...? [language]" 

In this case, we would need the word "France" to remember to later on predict the word "French" at the end of the sentence. RNNs will have problems to still refer to the word "France" although they consider the previous timesteps in the current one. This problem refers to the so called vanishing (or exploding) gradient problem which is captured in the formulas below. In practice this means that crucial information of very far back timesteps do not have an influence on the current or future timesteps anymore or that irrelevant information gets too much influence.
 
<img src="pics/vanishing_gradient.png" alt="vanishing_gradient" style="width: 600px;"/>
<br>

[(Olah, 2015)](http://colah.github.io/posts/2015-08-Understanding-LSTMs)


During backpropagation through time the gradient of the loss function can be represented as follows.

$$\frac{\partial J_t}{\partial W_{in}} =\frac{\partial J_t}{\partial y_{t}}\frac{\partial y_t}{\partial h_{t}}\frac{\partial h_t}{\partial h_{t-1}}
\frac{\partial h_{t-1}}{\partial h_{t-2}}\space\space ...\frac{\partial h_{0}}{\partial W_{in}}$$

It can be shown that as the gap between the timesteps gets bigger the product of the gradients of the hidden states w.r.t. the previous states
$$\frac{\partial h_t}{\partial h_{t-1}}\frac{\partial h_{t-1}}{\partial h_{t-2}}\frac{\partial h_{t-2}}{\partial h_{t-3}}\space\space...$$
<br>
gets longer and longer and a lot of very small numbers are multiplied. This can also be seen in the following equation. Since the weights $W_in$ are sampled from a standard normal distribution this term will mostly be smaller than one. Similarly, the absolute value of the tanh will also be between zero and one. [(Suresh, 2017)](http://introtodeeplearning.com/2017/Sequence%20Modeling.pdf)


$$\frac{\partial h_t}{\partial h_{t-1}}
= W_{in}^T diag[tanh^{ `}(W_{in}+W_h x_j)]$$



### Literature Review

As suggested by [(Fischer et. al, 2017)]( https://www.econstor.eu/bitstream/10419/157808/1/886576210.pdf) one can make good use of an LSTM network for financial market prediction since it is a state-of-the art technique for sequence learning. Even though this is not the same domain as our crime use case, the authors of the paper still demonstrate that LSTM networks can extract relevant information from the underlying noisy data. The LSTM network was primarily introduced by [(Hochreiter and Schmidhuber, 1997)](https://www.bioinf.jku.at/publications/older/2604.pdf) to overcome the vanishing gradient problem and to learn long-term dependencies since the simple RNN is not capable of doing that. We know that predicting crime rates is not the typical use case for introducing LSTM and GRU networks. Nevertheless, our crime data is also sequential since we are looking at a certain period of time to predict the future crime incidences and in addition, we expect to need the capability of learning long-term dependencies. Furthermore, there have already been some successful approaches forecasting crime with LSTM networks [(Stec et. al, 2018)](https://arxiv.org/pdf/1806.01486.pdf). The Gated Recurrent Unit network, which is also a Recurrent Neural Network, is also getting more and more popular for the task of prediction when having sequential data. It was only introduced a few years ago [(Cho et. al, 2014)](https://arxiv.org/pdf/1406.1078.pdf ) and it has been shown that the GRU performs similar or even better, depending on the data and the setup. For instance, traffic flow has been predicted by an LSTM and a GRU network whereby the GRU performed slightly better than the LSTM model [(Fu et. al,  2016)](https://www.researchgate.net/profile/Li_Li240/publication/312402649_Using_LSTM_and_GRU_neural_network_methods_for_traffic_flow_prediction/links/5c20d38d299bf12be3971696/Using-LSTM-and-GRU-neural-network-methods-for-traffic-flow-prediction.pdf). That is why usually both models are built to see which one performs better in the given use case
[(Nguyen, 2018)](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21).

### Advanced RNNs: LSTMs and GRUs

<img src="pics/overview_RNN_LSTM_GRU.png" alt="LSTM" style="width: 800px;"/>

[(Tch, 2017)](https://towardsdatascience.com/the-mostly-complete-chart-of-neural-networks-explained-3fb6f2367464)

As already suggested, there are more advanced solutions than the simple RNN for the prediction of the next step in a sequence. To overcome the problem of the vanishing (and exploding) gradient, i.e. having crucial information in a very far back timestep that might vanish or irrelevant information might happen to have too much influence on the current time step, we suggest to use LSTM (Long Short-Term Memory) and the GRU (Gated Recurrent Unit) networks. In the following, we will tackle these popular approaches. Instead of the RNN cells the LSTM and GRU have more complex cells in the hidden states. The idea is to control the information flow and regulate what is passed through and what not and decide what the relevant information is from the current as well as the previous timestep. This is done by the different gates in the gated cells. 

### LSTM Networks - Long Short-Term Memory Networks

<img src="pics/LSTM.png" alt="LSTM" style="width: 800px;"/>

The LSTM is a special type of RNN which can learn long-term dependencies. Instead of having only one tanh layer as in the RNN, more layers as can be seen as the yellow rectangles. The LSTM has 3 gates which are represented as the sigmoid layers. The output of sigmoid function is alsways something in between zero and one, i.e. to decide if information is let through or not. We will refer to Christopher Olah's graphics and explanations in this part [(Olah, 2015)](http://colah.github.io/posts/2015-08-Understanding-LSTMs). Let us walk you through the complex LSTM cell step by step.

<img src="pics/lstm_core_idea.png" alt="LSTM" style="width: 400px;"/>

There is always a horizontal line running through all cells between the previous cell state $C_t-1$ and the current cell state $C_t$ and in between there happen a few interactions. A very important fact about the LSTM cell is that there is the cell state on the one hand which is often referred to as a memory. On the other hand, there is also a hidden state which is often referred to as the output. One can maybe think of the hidden state $h_t$ as a filtered version of the cell state, whereas the cell state has the ability to hold on to information that is not necessary for the current output. For instance, when we want to output the next word, we might not need all the information of this word to output but maybe we want to save the information of this word, such as if it is singular or plural or a noun or a verb, to predict words in a future timestep. 
Therefore, the cell state can hold on to information that might be useful in upcoming timesteps. 

The key things for now are that we have three gates in the LSTM cell and that the hidden state is separate from the cell state. Furthermore, by the gates information can be removed or added from the cell state.

In [3]:
# Initialize sample data and weights

import numpy as np

x = np.array([0.2, 0.3, 0.4])
y = 7.0

Wi = np.array([[3.1], [0.1]])
Ui = np.array([[1.5,2.6], [2.1,0.2]])

Wf = np.array([[2.3], [0.2]])
Uf = np.array([[3.6,4.1], [1.0,0.9]])

Wo = np.array([[0.1], [3.1]])
Uo = np.array([[0.1,0.9], [0.7,4.3]])

Wc = np.array([[0.2], [0.4]])
Uc = np.array([[1.8,3.6], [4.7,2.9]])

w = np.array([2.0,4.0])

h0 = np.array([0.0,0.0])

### The Forget Gate

<img src="pics/lstm_forget.png" alt="LSTM" style="width: 700px;"/>

First, there is a forget gate which decides what information to forget or even to leave everything through as it is.
As input the output of the previous cell $h_t-1$ and input of the current timestep are fed into the cell. The output will be something in between zero and one, i.e. one means that everything is kept from the previous timestep whereas zero means that everything from the previous timestep will be forgotten.

In [4]:
inner_f1 = np.dot(Wf,x[[0]])+ np.dot(Uf,h0)

f1 = 1/(1+np.exp(-inner_f1))
f1

array([0.61301418, 0.50999867])

### The Input Gate

<img src="pics/lstm_input.png" alt="LSTM" style="width: 700px;"/>


In the next step, it will be decided selectively what information to update in the cell state. This is done in two steps. First, it is determined which values to update and second, the vector with the values of the new candidates for the cell state is calculated.

In [5]:
inner_i1 = np.dot(Wi,x[[0]]) + np.dot(Ui,h0)

i1 = 1/(1+np.exp(-inner_i1))
i1

array([0.65021855, 0.50499983])

In [6]:
inner_c1_tilde = np.dot(Wc, x[[0]]) + np.dot(Uc, h0)

c1_tilde = np.tanh(inner_c1_tilde)
c1_tilde

array([0.03997868, 0.07982977])

### The Update of the Cell State

<img src="pics/lstm_new_cell_state.png" alt="LSTM" style="width: 600px;"/>

Now, the new cell state can be computed as follows. The cell state of the previous timestep is multipliead by the values we want to forget, i.e. this first term of the equation gives us the part, that we want to remember from the previous timestep. Then, one adds the proportion determined by the input gate (of what we want to update) of the values of the new candidates. Basically, this is about how much is kept from the previous cell state and how much we take from the candidates for the new cell state to actually calculate the cell state of timestep t. 

In [7]:
c0 = np.array([0.0,0.0])

c1 = f1 * c0 + i1 * c1_tilde
c1

array([0.02599488, 0.04031402])

### The Output Gate

<img src="pics/lstm_output.png" alt="LSTM" style="width: 600px;"/>

So what is the ouput of the cell? Basically, it is a filtered version of the cell state. The sigmoid layer (the output gate) filters what parts of the cell state will be included in the output $h_t$. The cell state is put through a tanh layer and ultimately multiplied by the values of the output gate. Thus, the output is a filtered version of the cell state.

In [8]:
# outpute gate
inner_o1 = np.dot(Wo, x[[0]]) + np.dot(Uo, h0)

o1 = 1/(1+np.exp(-inner_o1))
o1

array([0.50499983, 0.65021855])

In [9]:
h1 = o1 * np.tanh(c1)
h1

array([0.01312445, 0.02619873])

Now, that we know how to calculate all the values of one time step in an LSTM cell, we can just plug in the new updated values of the input $x_t$, the hidden state $h_{t-1}$ and the cell state $c_t$ for the second and third time step into the same equations. Usually, one would implement this in a loop, but for the purpose of demonstration we show you the steps in detail.

### Second time step

In [10]:
# forget gate 
inner_f2 = Wf.dot(x[[1]])+ Uf.dot(h1)
f2 = 1/(1+np.exp(-inner_f2))

# input gate
inner_i2 = Wi.dot(x[[1]]) + Ui.dot(h1)
i2 = 1/(1+np.exp(-inner_i2))

inner_c2_tilde = Wc.dot(x[[1]]) + Uc.dot(h1)
c2_tilde = np.tanh(inner_c2_tilde)

# update of the cell state
c2 = f2 * c1 + i2 * c2_tilde

# output gate
inner_o2 = Wo.dot(x[[1]]) + Uo.dot(h1)
o2 = 1/(1+np.exp(-inner_o2))

h2 = o2 * np.tanh(c2)
h2

array([0.07524102, 0.11116973])

### Third time step

In [11]:
# forget gate 
inner_f3 = Wf.dot(x[[2]])+ Uf.dot(h2)
f3 = 1/(1+np.exp(-inner_f3))

# input gate
inner_i3 = Wi.dot(x[[2]]) + Ui.dot(h2)
i3 = 1/(1+np.exp(-inner_i3))

inner_c3_tilde = Wc.dot(x[[2]]) + Uc.dot(h2)
c3_tilde = np.tanh(inner_c3_tilde)

# update of the cell state
c3 = f3 * c2 + i3 * c3_tilde

# output gate
inner_o3 = Wo.dot(x[[2]]) + Uo.dot(h2)
o3 = 1/(1+np.exp(-inner_o3))

h3 = o3 * np.tanh(c3)
h3

array([0.28170128, 0.37065888])

### Prediction

After running through all the time steps, we predict our target variable.

In [12]:
y_ = w.dot(h3)
y_

2.046038096901425

### GRU Networks - Gated Recurrent Unit Networks

<img src="pics/GRU.png" alt="GRU" style="width: 600px;"/>

[(Olah, 2015)](http://colah.github.io/posts/2015-08-Understanding-LSTMs)

The GRU network is a variation of the LSTM network and looks similar. Instead of having three gates, it only has two gates, the update and the reset gate which also work similar to the gates in the LSTM cell. 

In [13]:
# Initialize sample data and weights

x = np.array([0.2, 0.3, 0.4])
y = 7.0

Wz = np.array([[0.1], [3.1]])
Uz = np.array([[0.1,4.1], [0.2,1.0]])

Wr = np.array([[2.3], [0.5]])
Ur = np.array([[1.3,7.1], [9.1,4.5]])

Wh = np.array([[0.2], [0.9]])
Uh = np.array([[1.5,2.6], [1.8,3.6]])

w = np.array([2.0,4.0])

As input, the cell gets the input of the current timestep and the previous output $h_t-1$. An important difference to the LSTM is that there is no separate cell state: the hidden state and cell state are kind of "merged" so that there only remains a hidden state $h_t$.

### Update Gate

In the update gate $z_t$ it is determined what values to update from the past and the reset gate decides what information should be reset, i.e. removed from the past.

In [14]:
h0 = np.array([0.0,0.0])

z1 = 1/(1+np.exp(-(Wz.dot(x[[0]])+ Uz.dot(h0))))
z1

array([0.50499983, 0.65021855])

### Reset gate

In [15]:
r1 = 1/(1+np.exp(-(Wr.dot(x[[0]])+ Ur.dot(h0))))
r1

array([0.61301418, 0.52497919])

### New candidates for current hidden state

The new candidates for the current cell state are calculated by taking information from the reset gate. 

In [16]:
h1_tilde = np.tanh(Wh.dot(x[[0]])+ Uh.dot(r1* h0))
h1_tilde

array([0.03997868, 0.17808087])

### New hidden state: the output

The new output $h_t$ is defined by the proportion $1-z_t$ of the previous hidden state $h_t-1$ and by the proportion $z_t$ of the new candidates for the current hidden state.

In [17]:
h1 = (1-z1)*h0 + (z1*h1_tilde)
h1

array([0.02018923, 0.11579148])

Again, we plug in all the updated values for the next time steps into the same equations like we did for the LSTM forward pass.

### Second time step

In [18]:
z2 = 1/(1+np.exp(-(Wz.dot(x[[1]])+ Uz.dot(h1))))

r2 = 1/(1+np.exp(-(Wr.dot(x[[1]])+ Ur.dot(h1))))

h2_tilde = np.tanh(Wh.dot(x[[1]])+ Uh.dot(r2* h1))

h2 = (1-z2)*h1+ (z2*h2_tilde)
h2

array([0.18717827, 0.42379445])

### Third time step

In [19]:
z3 = 1/(1+np.exp(-(Wz.dot(x[[2]])+ Uz.dot(h2))))

r3 = 1/(1+np.exp(-(Wr.dot(x[[2]])+ Ur.dot(h2))))

h3_tilde = np.tanh(Wh.dot(x[[2]])+ Uh.dot(r3* h2))

h3 = (1-z3)*h2+ z3*h3_tilde
h3

array([0.79220277, 0.8899337 ])

### Prediction

After running through all the time steps, we predict our target variable.

In [20]:
y_ = w.dot(h3) 
y_

5.144140350766751